# Симплекс-метод

В переменной df можно задавать любые ограничения и значения для исходной матрицы.

In [1]:
import pandas as pd
import numpy as np

epoch = 5

df = pd.DataFrame({ 'Z' : [-1,-1,-1,0],
                    'X4': [1,-1,2,8],
                    'X5': [2,-3,-1,1]},
                    index = ['X1','X2','X3','C'])
print("Начальная симплекс-таблица")
df = df.T 
df

Начальная симплекс-таблица


,X1,X2,X3,C
Z,-1,-1,-1,0
X4,1,-1,2,8
X5,2,-3,-1,1


In [2]:
def simplex_method(df):
    for i in range(epoch):
        print("Итерация {}".format(i+1))
        
        pivot_col = df.loc['Z'].drop('C').idxmin()
        pivot_ind = df[pivot_col].drop('Z').idxmax()
        pivot_num = max(df[pivot_col])

        df.loc[pivot_ind] /= pivot_num
        df.rename(columns={pivot_col:pivot_ind},index={pivot_ind:pivot_col},inplace=True)
        pivot_ind,pivot_col = pivot_col,pivot_ind
        df_ans = df.copy()
        display(df)

        
        df_piv_col = df.loc[pivot_ind].drop(pivot_col)
        df_piv_ind = df[pivot_col].drop(pivot_ind)
        sweep_df = df.drop(pivot_col, axis=1, inplace=True)       
        sweep_df = df.drop(pivot_ind)     

        
        mat_x1 = np.array([df_piv_ind.T for i in range(len(df_piv_col))]).T
        mat_x2 = np.diag(df_piv_col)
        mat_y = -np.dot(mat_x1,mat_x2)

        sweep_df += mat_y
        df_ans = pd.concat([df_ans,sweep_df], sort=True).fillna(0)
        
        grouped = df_ans.groupby(level=0)
        df_ans = grouped.last()

        display(df_ans)
        
        if sum(df_ans.loc['Z'] < 0) == 0:
            break

        df = df_ans

In [3]:
simplex_method(df)
print("Оптимальное решение")

Итерация 1


,X5,X2,X3,C
Z,-1.0,-1.0,-1.0,0.0
X4,1.0,-1.0,2.0,8.0
X1,1.0,-1.5,-0.5,0.5


,C,X2,X3,X5
X1,0.5,-1.5,-0.5,1.0
X4,7.5,0.5,2.5,0.0
Z,0.5,-2.5,-1.5,0.0


Итерация 2


,C,X4,X3,X5
X1,0.5,-1.5,-0.5,1.0
X2,15.0,1.0,5.0,0.0
Z,0.5,-2.5,-1.5,0.0


,C,X3,X4,X5
X1,23.0,7.0,0.0,1.0
X2,15.0,5.0,1.0,0.0
Z,38.0,11.0,0.0,0.0


Оптимальное решение
